In [106]:
import random
N = 32 # word length
M = 4  # num of words in the key
T = 27 # num of rounds necessary
j = 3
a=8
b=3

z = [0b01100111000011010100100010111110110011100001101010010001011111,
       0b01011010000110010011111011100010101101000011001001111101110001,
       0b11001101101001111110001000010100011001001011000000111011110101,
       0b11110000101100111001010001001000000111101001100011010111011011,
       0b11110111001001010011000011101000000100011011010110011110001011]
%run utilities.ipynb

In [107]:
def speck_round(x,y,round_key):
    k = round_key
    tmp = right_shift(x,a)
    outX = add_modulo(tmp,y,N) ^ k
    outY = left_shift(y,b) ^ outX
    return outX,outY

def inv_speck_round(x,y,round_key):
    k = round_key
    outY = right_shift(x^y,b)
    tmp = add_modulo(x ^ k,-outY,N)
    outX = left_shift(tmp,a)
    return outX,outY

def speck_encrypt_block(x,y,key):
    for i in range(T):
        x,y = speck_round(x,y,key[i])
    return x,y

def speck_decrypt_block(x,y,key):
    for i in reversed(range(T)):
        x,y = inv_speck_round(x,y,key[i])
    return x,y

def speck_encrypt(plain_integer,master_key):
    cipher_integer = []
    key = speck_keygen(master_key)
    for elem in plain_integer:
        cipher_integer.append((speck_encrypt_block(elem[0],elem[1],key )))
    return cipher_integer    

def speck_decrypt(cipher_integer,master_key):
    decrypt_integer = []
    key = speck_keygen(master_key)
    for elem in cipher_integer:
        decrypt_integer.append((speck_decrypt_block(elem[0],elem[1],key)))
    return decrypt_integer

def speck_keygen(master_key):
    master_key_array = split_4N(master_key)
    k = []
    l = []
    k+=master_key_array[0:1]
    l+= master_key_array[1:]
    for i in range(T-1):
        tmp = add_modulo(k[i],right_shift(l[i],a),N)
        tmp = tmp^i
        l.append(tmp)
        tmp = left_shift(k[i],b) ^l[i+M-1]
        k.append(tmp)
    
    return k
        


In [108]:
plain_text = "hello, this is secret message"
key = random.getrandbits(128)
plain_integer = string2input(plain_text)
cipher_integer = speck_encrypt(plain_integer,key)
decrypt_integer = speck_decrypt(cipher_integer,key)
decrypted_text = output2string(decrypt_integer)
print(decrypted_text)

hello, this is secret message


In [111]:
#open text file in read mode
text_file = open("loremIpsum", "r")
#read whole file to a string
plain_text = text_file.read()
key = random.getrandbits(128)
plain_integer = string2input(plain_text)

In [112]:
def test_encrypt():
    global plain_text
    global cipher_integer
    cipher_integer = speck_encrypt(plain_integer,key)

In [113]:
time test_encrypt()

CPU times: user 523 ms, sys: 4.64 ms, total: 528 ms
Wall time: 531 ms


In [114]:
def test_decrypt():
    global decrypt_integer
    global decrypted_text
    decrypt_integer = speck_decrypt(cipher_integer,key)

In [115]:
time test_decrypt()

CPU times: user 572 ms, sys: 42 ms, total: 614 ms
Wall time: 652 ms
